In [1]:
import os
from pathlib import Path
from typing import Any
from pydantic import Field, BaseModel
import httpx

from grasp_agents import (
    BaseTool,
    LLMAgent,
    RunContext,
    ImageData,
    Printer,
    print_event_stream,
    ParallelProcessor,
)
from grasp_agents.runner import Runner
from grasp_agents.typing.events import ProcPacketOutEvent
from grasp_agents.openai import OpenAILLM, OpenAILLMSettings
from grasp_agents.litellm import LiteLLM, LiteLLMSettings
from grasp_agents.workflow.sequential_workflow import SequentialWorkflow
from grasp_agents.cloud_llm import APIProvider
from grasp_agents.rate_limiting import RateLimiter

from grasp_agents.telemetry.traceloop import init_traceloop

# Optional: enable LLM observability with Phoenix
from grasp_agents.telemetry.phoenix import init_phoenix

In [2]:
PACKAGE_DIR = Path.cwd()

Paths to images used in the demo

In [3]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

Utils

In [4]:
def print_single_output(out: Any) -> None:
    print(f"\n<final answer>\n{out.payloads[0]}\n</final answer>")

**[Optional] Observability with Arize Phoenix (deployed locally via Docker)**

Start Phoenix locally:
```bash
cd ./phoenix
docker compose up -d  
docker compose logs -f phoenix
export PHOENIX_COLLECTOR_HTTP_ENDPOINT=http://localhost:6006/v1/traces # from docker-compose.yml
export TELEMETRY_PROJECT_NAME_KEY="openinference.project.name" # required by Phoenix
```

Open `http://localhost:6006/` (or the port you set in `docker-compose.yml`) in your browser to access the Phoenix UI.

Initialize telemetry

In [2]:
!export TELEMETRY_PROJECT_NAME_KEY

In [5]:
# Use Traceloop to produce spans (without sending them anywhere yet)
init_traceloop(project_name="agents-demo-10")

# Use Phoenix as the backend and UI for telemetry
# Alternatively, any OpenTelemetry-compatible backend can be used instead
init_phoenix(batch=False, use_litellm_instr=True, use_llm_provider_instr=False)

Traceloop exporting traces to a custom exporter

Metrics are disabled


## Simple generation with validated outputs

Output type validation

In [5]:
# list[int] is the output type used to validate the output
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="openai/gpt-4.1",
        llm_settings=OpenAILLMSettings(logprobs=True),
        max_response_retries=2,
    ),
)

# This initialises printer and usage tracker
ctx = RunContext[None](printer=Printer())

In [6]:
# Code block delimiters are stripped from the output
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)

<chatbot> [eeb616_chatbot]
<input>
Output a list of 3 integers from 0 to 10 as a python array, no talking
</input>

<chatbot> [eeb616_chatbot]
<response>
[
  7,
  2,
  9
]
</response>

------------------------------------
I/O/R/C tokens: 30/9/-/-


<final answer>
[7, 2, 9]
</final answer>


In [ ]:
ctx.usage_tracker.usages

Completion data (e.g. log probs) per agent can be accessed via RunContext:

In [ ]:
# ctx.completions

Streaming

In [7]:
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=LiteLLM(
        model_name="claude-sonnet-4-20250514",
        llm_settings=LiteLLMSettings(reasoning_effort=None),
    ),
    stream_llm_responses=True,
)
ctx = RunContext[None](printer=None)

In [8]:
async for event in print_event_stream(
    chatbot.run_stream(
        "Output a list of 30 integers from 0 to 10 as a python array. "
        "No code or talking.",
        ctx=ctx,
    )
):
    if isinstance(event, ProcPacketOutEvent):
        out = event.data


<chatbot> [8b629b_chatbot]
<input>
Output a list of 30 integers from 0 to 10 as a python array. No code or talking.
</input>

<chatbot> [8b629b_chatbot]
<response>
[3, 7, 1, 9, 4, 2, 8, 5, 0, 6, 10, 2, 7, 4, 9, 1, 5, 8, 3, 0, 6, 10, 4, 7, 2, 9, 1, 5, 8, 3]
</response>

<chatbot> [8b629b_chatbot]
<processor output>
[
  3,
  7,
  1,
  9,
  4,
  2,
  8,
  5,
  0,
  6,
  10,
  2,
  7,
  4,
  9,
  1,
  5,
  8,
  3,
  0,
  6,
  10,
  4,
  7,
  2,
  9,
  1,
  5,
  8,
  3
]
</processor output>


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[3, 7, 1...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='I am a l...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='I said: ...er_specific_fields=None), inp

In [ ]:
ctx.usage_tracker.usages

Output type validation with structured outputs

In [9]:
# Some providers (e.g. `openai` and `gemini`) support structured outputs.
# With the OpenAI API, this will require a Pydantic model to validate the output.

from enum import StrEnum


class Selector(StrEnum):
    A = "A"
    B = "B"


class Response(BaseModel):
    result: list[int] = Field(..., description="3 random integers")
    value: Selector = Field(..., description="Choose a value randomly")


chatbot = LLMAgent[None, Response, None](
    name="chatbot",
    response_schema=Response,
    llm=LiteLLM(
        model_name="gpt-4.1",
        llm_settings=LiteLLMSettings(),
        apply_response_schema_via_provider=True,
    ),
)

# By default, response_schema is set to the output type of the agent (Response)
# In some cases, you may want to set it to a different type, e.g. when using
# custom output parsing.

ctx = RunContext[None](printer=Printer())

In [10]:
out = await chatbot.run("start", ctx=ctx)
print_single_output(out)

<chatbot> [b559dd_chatbot]
<input>
start
</input>

<chatbot> [b559dd_chatbot]
<response>
{
  "result": [
    8,
    14,
    5
  ],
  "value": "A"
}
</response>

------------------------------------
I/O/R/C tokens: 106/13/-/-


<final answer>
result=[8, 14, 5] value=<Selector.A: 'A'>
</final answer>


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='{"result...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


# Chat with images

In [11]:
chatbot = LLMAgent[None, str, None](
    name="chatbot",
    llm=LiteLLM(
        model_name="gemini/gemini-2.5-flash",
        llm_settings=LiteLLMSettings(reasoning_effort="disable"),
    ),
)
ctx = RunContext[None](printer=Printer())

In [12]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print_single_output(out)

<chatbot> [ccacbb_chatbot]
<input>
Where are you headed, stranger?
</input>

<chatbot> [ccacbb_chatbot]
<response>
I am a large language model, trained by Google. I am not a person and therefore do not have a physical body or a destination. I exist only as a computer program.
</response>

------------------------------------
I/O/R/C tokens: 8/36/-/-


<final answer>
I am a large language model, trained by Google. I am not a person and therefore do not have a physical body or a destination. I exist only as a computer program.
</final answer>


In [13]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [d698ce_chatbot]
<input>
What did you just say, exactly?
</input>

<chatbot> [d698ce_chatbot]
<response>
I said: "I am a large language model, trained by Google. I am not a person and therefore do not have a physical body or a destination. I exist only as a computer program."
</response>

------------------------------------
I/O/R/C tokens: 54/40/-/-


<final answer>
I said: "I am a large language model, trained by Google. I am not a person and therefore do not have a physical body or a destination. I exist only as a computer program."
</final answer>


In [14]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print_single_output(out)

<chatbot> [af9f3c_chatbot]
<input>
What's in this image?
<ENCODED_IMAGE>
</input>

<chatbot> [af9f3c_chatbot]
<response>
The image shows a mathematical expression written in white text on a dark blue background. The expression is:

`7 * (5 + 15) / (2 * 5) - 3`
</response>

------------------------------------
I/O/R/C tokens: 361/42/-/-


<final answer>
The image shows a mathematical expression written in white text on a dark blue background. The expression is:

`7 * (5 + 15) / (2 * 5) - 3`
</final answer>


In [15]:
out = await chatbot.run("Go on", ctx=ctx)
print_single_output(out)

<chatbot> [f50f66_chatbot]
<input>
Go on
</input>

<chatbot> [f50f66_chatbot]
<response>
This mathematical expression can be evaluated using the order of operations (PEMDAS/BODMAS):

1.  **Parentheses/Brackets:**
    *   `(5 + 15)` evaluates to `20`
    *   `(2 * 5)` evaluates to `10`

2.  **Multiplication and Division** (from left to right):
    *   `7 * 20` evaluates to `140`
    *   `140 / 10` evaluates to `14`

3.  **Addition and Subtraction** (from left to right):
    *   `14 - 3` evaluates to `11`

So, the value of the expression `7 * (5 + 15) / (2 * 5) - 3` is `11`.
</response>

------------------------------------
I/O/R/C tokens: 407/183/-/-


<final answer>
This mathematical expression can be evaluated using the order of operations (PEMDAS/BODMAS):

1.  **Parentheses/Brackets:**
    *   `(5 + 15)` evaluates to `20`
    *   `(2 * 5)` evaluates to `10`

2.  **Multiplication and Division** (from left to right):
    *   `7 * 20` evaluates to `140`
    *   `140 / 10` evaluates to `

In [16]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print_single_output(out)

<chatbot> [0c52d9_chatbot]
<input>
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG
</input>

<chatbot> [0c52d9_chatbot]
<response>
The image shows a blue rounded rectangle with white text. It presents an "Arithmetic Expression" with given variable values and an equation to solve.

Specifically:
*   **Title:** "Arithmetic Expression:"
*   **Variable values:** `a=2, b=3, c=4`
*   **Equation:** `Z = a + b - (a * c)`
</response>

------------------------------------
I/O/R/C tokens: 504/79/-/349


<final answer>
The image shows a blue rounded rectangle with white text. It presents an "Arithmetic Expression" with given variable values and an equation to solve.

Specifically:
*   **Title:** "Arithmetic Expression:"
*   **Variable values:** `a=2, b=3, c=4`
*   **Equation:** `Z = a + b - (a * c)`
</final answer>


In [17]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [4dabc1_chatbot]
<input>
What was my first question, exactly?
</input>

<chatbot> [4dabc1_chatbot]
<response>
Your first question was: "Where are you headed, stranger?"
</response>

------------------------------------
I/O/R/C tokens: 199/13/-/743


<final answer>
Your first question was: "Where are you headed, stranger?"
</final answer>


In [14]:
ctx.usage_tracker.total_usage

Usage(input_tokens=2048, output_tokens=472, reasoning_tokens=None, cached_tokens=1199, cost=0.0013659700000000002)

# Parallel runs with retries and rate limiting

In [18]:
# Make the LLM generate text instead of integers occasionally
# to emphasise the need for retries

sys_prompt = """
You are a bad math student who always adds number {added_num} to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
"""

in_prompt = "What is the square of {num}?"


class RunArgs(BaseModel):
    added_num: int


class InputArgs(BaseModel):
    num: int


# Specifying int as the output type means that the agent will
# validate the output against this type.

student = LLMAgent[InputArgs, int, RunArgs](
    name="student",
    llm=LiteLLM(
        model_name="gpt-4.1",
        llm_settings=LiteLLMSettings(temperature=1.8),
        # This rate limit will be applied to parallel runs of the agent
        rate_limiter=RateLimiter(rpm=100),
    ),
    sys_prompt=sys_prompt,
    in_prompt=in_prompt,
    max_retries=2,
)


@student.add_system_prompt_builder
def system_prompt_builder(ctx: RunContext[RunArgs], **kwargs: Any) -> str:
    return student.sys_prompt.format(added_num=ctx.state.added_num)

In [20]:
in_args = [InputArgs(num=i) for i in range(10)]

In [21]:
ctx = RunContext[RunArgs](state=RunArgs(added_num=5), printer=Printer())

# Wrap the agent in a ParallelProcessor to run it in parallel
out = await ParallelProcessor(student).run(in_args=in_args, ctx=ctx)

print()
print(*[p for p in out.payloads], sep="\n")

<student> [6116f7_student_par/0]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [6116f7_student_par/0]
<input>
What is the square of 0?
</input>

<student> [6116f7_student_par/1]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [6116f7_student_par/1]
<input>
What is the square of 1?
</input>

<student> [6116f7_student_par/2]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [6116f7_student_par/2]
<input>
What is the square of 2?
</input>

<student> [6116f7_student_par/3]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer o

Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
9 + 5 = 14
Processor run failed [proc_name=student; call_id=6116f7_student_par/3] -> retrying (attempt 1):
Failed to validate LLM response:
9 + 5 = 14
Expected type: <class 'int'>


<student> [6116f7_student_par/3]
<input>
What is the square of 3?
</input>



/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='9 + 5 = ...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


<student> [6116f7_student_par/4]
<response>
21
</response>

------------------------------------
I/O/R/C tokens: 61/1/-/-

<student> [6116f7_student_par/5]
<response>
30
</response>

------------------------------------
I/O/R/C tokens: 61/1/-/-

<student> [6116f7_student_par/6]
<response>
41
</response>

------------------------------------
I/O/R/C tokens: 61/1/-/-

<student> [6116f7_student_par/7]
<response>
54
</response>

------------------------------------
I/O/R/C tokens: 61/1/-/-

<student> [6116f7_student_par/8]
<response>
69
</response>

------------------------------------
I/O/R/C tokens: 61/1/-/-



...

<student> [6116f7_student_par/9]
<input>
What is the square of 9?
</input>



/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Eighty-s...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
Three squared is 9, but I… I wouldn’t write 9. 
I’d add 5 and come up with 14. So my answer? 14
Processor run failed [proc_name=student; call_id=6116f7_student_par/3] -> retrying (attempt 2):
Failed to validate LLM response:
Three squared is 9, but I… I wouldn’t write 9

<student> [6116f7_student_par/3]
<input>
What is the square of 3?
</input>

<student> [6116f7_student_par/9]
<response>
86
</response>

------------------------------------
I/O/R/C tokens: 75/1/-/-



/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Three sq...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


<student> [6116f7_student_par/3]
<response>
14
</response>

------------------------------------
I/O/R/C tokens: 89/1/-/-


5
6
9
14
21
30
41
54
69
86


In [ ]:
ctx = RunContext[RunArgs](state=RunArgs(added_num=5))

# Do not stream granular LLM events here
stream = ParallelProcessor(student).run_stream(in_args=in_args, ctx=ctx)
async for event in print_event_stream(stream):
    pass


<student> [0083f1_student_par/0]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/0]
<input>
What is the square of 0?
</input>

<student> [0083f1_student_par/1]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/1]
<input>
What is the square of 1?
</input>

<student> [0083f1_student_par/2]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/2]
<input>
What is the square of 2?
</input>

<student> [0083f1_student_par/3]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer 


<student> [0083f1_student_par/5]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/5]
<input>
What is the square of 5?
</input>

<student> [0083f1_student_par/6]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/6]
<input>
What is the square of 6?
</input>

<student> [0083f1_student_par/7]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [0083f1_student_par/7]
<input>
What is the square of 7?
</input>

<student> [0083f1_student_par/8]
<system>
You are a bad math student who always adds number 5 to the correct result of the operation. 
Output a single integer 

Both ast.literal_eval and json.loads failed to parse the following JSON/Python string:
Six
Processor run failed [proc_name=student; call_id=0083f1_student_par/1] -> retrying (attempt 1):
Failed to validate LLM response:
Six
Expected type: <class 'int'>



<student> [0083f1_student_par/1]
<input>
What is the square of 1?
</input>


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Six', ro...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='9', role...: None}, annotations=[]), inp


<student> [0083f1_student_par/5]
<input>
What is the square of 5?
</input>


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='３０',...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='69', rol...: None}, annotations=[]), input_t


<student_par> [0083f1_student_par]
<processor output>
5
6
9
14
21
30
41
54
69
86
</processor output>


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='30', rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Great! S...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='to...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


# Reasoning agent loop with streaming

In [23]:
sys_prompt_react = """
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
"""

In [24]:
# Tool input must be a Pydantic model to infer the JSON schema used by the LLM APIs
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    reply: str
        The student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description

    async def run(self, inp: TeacherQuestion, **kwargs: Any) -> StudentReply:
        return input(inp.question)

In [26]:
class Problem(BaseModel):
    problem: str


teacher = LLMAgent[None, Problem, None](
    name="teacher",
    llm=LiteLLM(
        model_name="claude-sonnet-4-20250514",
        llm_settings=LiteLLMSettings(reasoning_effort="low"),
    ),
    tools=[AskStudentTool()],
    # react_mode=True, # use with non-reasoning models to enforce preamble/tool/response structure
    final_answer_as_tool_call=True,
    sys_prompt=sys_prompt_react,
    stream_llm_responses=True,
)

In [ ]:
ctx = RunContext[None]()

events = []
problem: Problem
async for event in print_event_stream(teacher.run_stream("start", ctx=ctx)):
    if isinstance(event, ProcPacketOutEvent):
        problem = event.data.payloads[0]
    events.append(event)


<teacher> [adec5c_teacher]
<system>
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
</system>

<teacher> [adec5c_teacher]
<input>
start
</input>

<teacher> [adec5c_teacher]
<thinking>
I need to help suggest an exciting statistics problem tailored to the student. To do this effectively, I should first learn about their background, interests, and preferences. I'll start by asking about their educational background and level of statistics knowledge.
</thinking>
<response>
I'd be happy to help you find an exciting statistics problem that's perfect for you! Let me start by learning a bit about your background.
</response>
<tool call> ask_student [toolu

/Users/iana/Documents/grasp-agents/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [field_name='tool_calls', input_value={'function': {'arguments'...r8', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(



<teacher> [adec5c_teacher]
<response>
Great! Since you're at the master's level, you likely have solid statistical foundations. Now let me learn about your interests to tailor a problem that will genuinely excite you.
</response>
<tool call> ask_student [toolu_01XQtGUbkdHo43dosZRF7TJP]
{"question": "What are your main interests or areas of study? For example, are you interested in business, psychology, sports, technology, healthcare, social sciences, or something else entirely?"}
</tool call>


In [ ]:
problem

# Sequential workflow 

In [4]:
# Input arguments are passed to the agent dynamically (e.g. by other agents)
from grasp_agents.typing.content import Content


# Global state is used to store data that is shared between runs of the agent.
class State(BaseModel):
    b: int
    c: int


class AddInputArgs(BaseModel):
    a: int = Field(..., description="First number to add.")


class AddResponse(BaseModel):
    a_plus_b: int


add_in_prompt = "Add {a} and {b}. Your only output is the resulting number."


add_agent = LLMAgent[AddInputArgs, AddResponse, State](
    name="add_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    in_prompt=add_in_prompt,
    # Reset message history to system prompt (if provided) before each run
    reset_memory_on_run=True,
    stream_llm_responses=True,
)


@add_agent.add_input_content_builder
def build_input_content_impl(
    in_args: AddInputArgs, ctx: RunContext[State], call_id: str
) -> Content:
    return Content.from_formatted_prompt(
        add_agent.in_prompt, a=in_args.a, b=ctx.state.b
    )


@add_agent.add_output_parser
def parse_output_impl(
    final_answer: str,
    *,
    in_args: AddInputArgs | None = None,
    ctx: RunContext[State],
    call_id: str,
) -> AddResponse:
    return AddResponse(a_plus_b=int(final_answer.strip()))

NameError: name 'BaseModel' is not defined

In [ ]:
class MultiplyResponse(BaseModel):
    c_a_plus_b: int


multiply_in_prompt = (
    "Multiply {a_plus_b} by {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, State](
    name="multiply_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    in_prompt=multiply_in_prompt,
    reset_memory_on_run=True,
    stream_llm_responses=True,
)


# Need a custom input content maker to use the global state
@multiply_agent.add_input_content_builder
def build_input_content_impl(
    in_args: AddResponse, ctx: RunContext[State], call_id: str
) -> Content:
    return Content.from_formatted_prompt(
        multiply_agent.in_prompt, a_plus_b=in_args.a_plus_b, c=ctx.state.c
    )


@multiply_agent.add_output_parser
def parse_output_impl(
    final_answer: str,
    *,
    in_args: AddResponse | None = None,
    ctx: RunContext[State],
    call_id: str,
) -> MultiplyResponse:
    return MultiplyResponse(c_a_plus_b=int(final_answer.strip()))

In [ ]:
seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, State](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)

In [ ]:
state = State(b=3, c=6)
ctx = RunContext[State](state=state, printer=Printer())

In [ ]:
out = await seq_agent.run(in_args=AddInputArgs(a=2), ctx=ctx)

# out = await ParallelProcessor(seq_agent).run(
#     in_args=[AddInputArgs(a=2), AddInputArgs(a=3)], ctx=ctx
# )

# Agents as tools

When agents are used as tools, their `in_args` become the tool inputs.

This is how one can implement a manager + helpers architecture.

In [1]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)

NameError: name 'seq_agent' is not defined

The JSON schema of `in_args` is preserved:

In [ ]:
seq_tool.in_type.model_json_schema()

In [ ]:
await seq_tool(a=15, ctx=ctx)

Stream from the tool

In [ ]:
state = State(b=3, c=6)
ctx = RunContext[State](state=state)

seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, State](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)
stream = seq_tool.run_stream(AddInputArgs(a=15), ctx=ctx)

async for event in print_event_stream(stream):
    pass

# Teacher / students

A more advanced example of multi-agent debate, where agents communicate using the actor model.

Communication schemas

In [2]:
from collections.abc import Sequence
from typing import Literal


TeacherRecipient = Literal["*END*", "teacher", "student1", "student2"]


# Teacher can choose which students to send the message to
class TeacherExplanation(BaseModel):
    explanation: str
    selected_recipients: Sequence[TeacherRecipient] = Field(
        default_factory=list[TeacherRecipient],
        description="Recipients selected by the teacher.",
    )


# Students can only ask questions to the teacher
class StudentQuestion(BaseModel):
    question: str = Field(
        ...,
        description="The question to ask the teacher.",
    )

NameError: name 'BaseModel' is not defined

#### Teacher

In [ ]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named 'student1') and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
To indicate to whom you are addressing your message, you must specify the recipients as a list of selected student names. 
When students have no more questions, finish the conversation with a SINGLE message with a SINGLE recipient called *END*. 
Do not produce multiple "thanks" or "goodbye" messages, just a single one.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    name="teacher",
    llm=LiteLLM(model_name="gpt-4o", apply_response_schema_via_provider=True),
    sys_prompt=teacher_sys_prompt,
    # need to specify allowed recipients to choose from
    recipients=["*END*", "student1", "student2"],
)


@teacher.add_recipient_selector
def select_recipients_impl(
    output: TeacherExplanation, **kwargs: Any
) -> Sequence[TeacherRecipient] | None:
    return output.selected_recipients

#### Students

In [ ]:
student_sys_prompts = [
    """
You are a 4-year old child trying to make sense of physics. 
Your name is <student1>.
Talk to the teacher to understand the topic.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 4-year old child.
You talk to the teacher only.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    student = LLMAgent[TeacherExplanation, StudentQuestion, None](
        name=name,
        llm=LiteLLM(model_name="gpt-4o", apply_response_schema_via_provider=True),
        sys_prompt=sys_prompt,
        recipients=["teacher"],
    )

    @student.add_output_parser
    def parse_output_impl(final_answer: str, **kwargs: Any) -> StudentQuestion:
        return StudentQuestion(question=f"<{name}>: " + str(final_answer))

    return student


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])

In [ ]:
ctx = RunContext[None](printer=Printer(color_by="agent"))
runner = Runner(entry_proc=teacher, procs=[teacher, student1, student2], ctx=ctx)
final_result = await runner.run("start")

Streaming

In [ ]:
runner = Runner(
    entry_proc=teacher, procs=[teacher, student1, student2], ctx=RunContext[None]()
)
events = []
async for event in print_event_stream(
    runner.run_stream(chat_inputs="start"), color_by="agent"
):
    events.append(event)

# Custom API providers and HTTP clients

In [ ]:
custom_provider = APIProvider(
    name="openrouter",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

http_client = httpx.AsyncClient(
    timeout=httpx.Timeout(10),
    limits=httpx.Limits(max_connections=10),
)

chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="deepseek/deepseek-r1-0528",
        api_provider=custom_provider,
        http_client=http_client,
    ),
)


ctx = RunContext[None](printer=Printer())
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)